# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [7]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [12]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [15]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

29.59

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [50]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [51]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data.status_code)
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
        ignore_index = True
        )
#         print(symbol)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,144.371,47.81,N/A
1,AAL,24.688,-1.71,N/A
2,AAP,199.640,21.45,N/A
3,AAPL,131.080,29.59,N/A
4,ABBV,117.700,39.7,N/A
...,...,...,...,...
500,YUM,123.000,33.01,N/A
501,ZBH,162.390,58.12,N/A
502,ZBRA,530.230,42.92,N/A
503,ZION,58.386,7.15,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [54]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace=True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [56]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [57]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [61]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,593.590,4.31,33
1,CINF,125.160,6.77,159
2,ZION,58.386,7.15,342
3,AFL,59.170,7.52,338
4,COO,375.620,8.3,53
5,HOLX,62.390,8.41,320
6,UNM,32.790,8.61,609
7,PGR,97.990,8.79,204
8,VIAC,42.000,9.4,476
9,CE,167.186,9.43,119


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [93]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [99]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
        ],
        index = rv_columns),
        ignore_index = True
        )

In [100]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,141.540,N/A,47.24,N/A,8.95,N/A,7.23,N/A,30.607858,N/A,14.028309,N/A,N/A
1,AAL,24.885,N/A,-1.71,N/A,-2.02,N/A,1.25,N/A,-3.979406,N/A,3.213572,N/A,N/A
2,AAP,204.267,N/A,21.8,N/A,3.67,N/A,1.21,N/A,10.763995,N/A,2.737044,N/A,N/A
3,AAPL,128.904,N/A,29.76,N/A,31.44,N/A,6.48,N/A,22.169589,N/A,16.502981,N/A,N/A
4,ABBV,115.780,N/A,39.81,N/A,14.77,N/A,4.03,N/A,12.781784,N/A,8.390287,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.090,N/A,32.14,N/A,-4.6,N/A,6,N/A,23.468663,N/A,10.687275,N/A,N/A
501,ZBH,163.730,N/A,59.63,N/A,2.75,N/A,4.75,N/A,28.435841,N/A,8.276718,N/A,N/A
502,ZBRA,507.650,N/A,44,N/A,11.5,N/A,5.94,N/A,32.018747,N/A,12.909382,N/A,N/A
503,ZION,59.592,N/A,7.26,N/A,1.3,N/A,2.73,N/A,5.880023,N/A,2.797093,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [104]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,252.050,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,295.180,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.13,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,29.250,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.820,N/A,14.86,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,64.850,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.210,N/A,33.56,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.982,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,201.460,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2596.040,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [111]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [113]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [163]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,32.845,1522,8.54,0.38,0.6343,0.16,0.5040,0.22,4.174346,0.16,0.489947,0.06,6.21782
1,MCK,197.353,253,-7.00,0.26,-1508.5000,0.02,0.1325,0.08,7.664094,0.54,2.711032,0.6,6.33663
2,AIV,7.715,6480,-84.60,0.08,2.1000,0.9,-34.3400,0.02,-44.001670,0.02,-55.362201,0.02,6.77228
3,L,59.510,840,-430.75,0.02,0.9056,0.22,1.0900,0.47,8.825861,0.7,1.085156,0.18,7.30693
4,AIG,55.878,894,-12.30,0.24,0.7548,0.2,1.0900,0.47,6.891653,0.36,1.090176,0.2,7.64356
5,HFC,37.000,1351,-39.97,0.12,1.1700,0.34,0.5359,0.24,9.101865,0.76,2.990405,0.68,9.42574
6,ABC,121.900,410,-6.72,0.28,-86.7000,0.04,0.1263,0.06,8.902390,0.72,4.408866,0.92,9.54455
7,AAL,24.885,2009,-1.71,0.3,-2.0200,0.12,1.2500,0.58,-3.979406,0.06,3.213572,0.72,10.0792
8,BA,254.870,196,-12.66,0.22,-8.4800,0.1,2.7100,0.94,-20.098002,0.04,-42.313877,0.04,10.495
9,ALL,136.573,366,11.94,0.6,1.7000,0.74,0.8626,0.38,2.607134,0.1,0.863092,0.12,10.5941


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [164]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,32.845,1522,8.54,0.38,0.6343,0.16,0.5040,0.22,4.174346,0.16,0.489947,0.06,0.196
1,MCK,197.353,253,-7.00,0.26,-1508.5000,0.02,0.1325,0.08,7.664094,0.54,2.711032,0.6,0.3
2,AIV,7.715,6480,-84.60,0.08,2.1000,0.9,-34.3400,0.02,-44.001670,0.02,-55.362201,0.02,0.208
3,L,59.510,840,-430.75,0.02,0.9056,0.22,1.0900,0.47,8.825861,0.7,1.085156,0.18,0.318
4,AIG,55.878,894,-12.30,0.24,0.7548,0.2,1.0900,0.47,6.891653,0.36,1.090176,0.2,0.294
5,HFC,37.000,1351,-39.97,0.12,1.1700,0.34,0.5359,0.24,9.101865,0.76,2.990405,0.68,0.428
6,ABC,121.900,410,-6.72,0.28,-86.7000,0.04,0.1263,0.06,8.902390,0.72,4.408866,0.92,0.404
7,AAL,24.885,2009,-1.71,0.3,-2.0200,0.12,1.2500,0.58,-3.979406,0.06,3.213572,0.72,0.356
8,BA,254.870,196,-12.66,0.22,-8.4800,0.1,2.7100,0.94,-20.098002,0.04,-42.313877,0.04,0.268
9,ALL,136.573,366,11.94,0.6,1.7000,0.74,0.8626,0.38,2.607134,0.1,0.863092,0.12,0.388


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [165]:
rv_dataframe.sort_values('RV Score', ascending= True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe 

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,32.845,1522,8.54,0.38,0.6343,0.16,0.5040,0.22,4.174346,0.16,0.489947,0.06,0.196
1,AIV,7.715,6480,-84.60,0.08,2.1000,0.9,-34.3400,0.02,-44.001670,0.02,-55.362201,0.02,0.208
2,BA,254.870,196,-12.66,0.22,-8.4800,0.1,2.7100,0.94,-20.098002,0.04,-42.313877,0.04,0.268
3,AIG,55.878,894,-12.30,0.24,0.7548,0.2,1.0900,0.47,6.891653,0.36,1.090176,0.2,0.294
4,MCK,197.353,253,-7.00,0.26,-1508.5000,0.02,0.1325,0.08,7.664094,0.54,2.711032,0.6,0.3
5,L,59.510,840,-430.75,0.02,0.9056,0.22,1.0900,0.47,8.825861,0.7,1.085156,0.18,0.318
6,STX,104.320,479,26.40,1,0.0000,0.14,0.0000,0.04,3.014944,0.12,1.451436,0.34,0.328
7,AAL,24.885,2009,-1.71,0.3,-2.0200,0.12,1.2500,0.58,-3.979406,0.06,3.213572,0.72,0.356
8,ALL,136.573,366,11.94,0.6,1.7000,0.74,0.8626,0.38,2.607134,0.1,0.863092,0.12,0.388
9,ABC,121.900,410,-6.72,0.28,-86.7000,0.04,0.1263,0.06,8.902390,0.72,4.408866,0.92,0.404


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [166]:
portfolio_input()

Enter the value of your portfolio:1000000


In [167]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,32.845,608,8.54,0.38,0.6343,0.16,0.5040,0.22,4.174346,0.16,0.489947,0.06,0.196
1,AIV,7.715,2592,-84.60,0.08,2.1000,0.9,-34.3400,0.02,-44.001670,0.02,-55.362201,0.02,0.208
2,BA,254.870,78,-12.66,0.22,-8.4800,0.1,2.7100,0.94,-20.098002,0.04,-42.313877,0.04,0.268
3,AIG,55.878,357,-12.30,0.24,0.7548,0.2,1.0900,0.47,6.891653,0.36,1.090176,0.2,0.294
4,MCK,197.353,101,-7.00,0.26,-1508.5000,0.02,0.1325,0.08,7.664094,0.54,2.711032,0.6,0.3
5,L,59.510,336,-430.75,0.02,0.9056,0.22,1.0900,0.47,8.825861,0.7,1.085156,0.18,0.318
6,STX,104.320,191,26.40,1,0.0000,0.14,0.0000,0.04,3.014944,0.12,1.451436,0.34,0.328
7,AAL,24.885,803,-1.71,0.3,-2.0200,0.12,1.2500,0.58,-3.979406,0.06,3.213572,0.72,0.356
8,ALL,136.573,146,11.94,0.6,1.7000,0.74,0.8626,0.38,2.607134,0.1,0.863092,0.12,0.388
9,ABC,121.900,164,-6.72,0.28,-86.7000,0.04,0.1263,0.06,8.902390,0.72,4.408866,0.92,0.404


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [172]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = "Value Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [174]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [175]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [177]:
writer.save()